In [2]:
# 进入工作目录
%cd ../

/home/heyilong/codes/chinese_medical_ner_submit/project/code


In [ ]:
# 读取预处理后的文本数据
train_path = "../data/train"
test_path = "../data/chusai"

from data_process.preprocess import PreProcess
from glob import glob

text_list = glob(train_path + "/*.txt") + glob(test_path + "*.txt")

texts = []
for txt_name in text_list:
    preprocess = PreProcess(txt_name)
    conll, info = preprocess.brat2conll()
    for sen in conll:
        text = "".join([c[0] for c in sen]).replace("[unused1]", " ")
        texts.append(text)



In [ ]:
# 读取tencent词向量中的所有word, 生成vocab
from tqdm import tqdm
vocab = []

i = 10
is_first = True
pbar = None
with open("../data/tencent_embedding/Tencent_AILab_ChineseEmbedding.txt", "r", encoding="utf8") as r:
    for line in r:
        ls = line.rstrip().split(" ", 1)
        if is_first:
            pbar = tqdm(total=int(ls[0]))
            is_first = False
            continue
        else:
            vocab.append(ls[0])
            pbar.update(1)

pbar.close()
vocab = sorted(vocab) # 按ascii排序，防止词id改变

with open("../data/tencent_embedding/vocab.txt", "w", encoding="utf8") as w:
    w.write("\n".join(vocab))

In [ ]:
from tqdm import tqdm
# from ruler.flashtext import KeywordProcessor
# simlpe_tencent_vocab = set()
# tree_trie = KeywordProcessor()
# tree_trie.add_keywords_from_list(vocab)

all_text = ""
for text in tqdm(texts):
    all_text+=text
simlpe_tencent_vocab = set()
for word in tqdm(vocab):
    if word in all_text:
        simlpe_tencent_vocab.add(word)

simlpe_tencent_vocab.add("</s>")
simlpe_tencent_vocab.add("unk")

In [ ]:
# 生成tencent简化版词向量
simlpe_tencent_embedding = []
is_first = True
pbar = None
with open("../data/tencent_embedding/Tencent_AILab_ChineseEmbedding.txt", "r", encoding="utf8") as r:
    for line in r:
        ls = line.rstrip().split(" ", 1)
        if is_first:
            pbar = tqdm(total=int(ls[0]))
            is_first = False
            continue
        else:
            if ls[0] in simlpe_tencent_vocab:
                simlpe_tencent_embedding.append(ls)
            pbar.update(1)
pbar.close()

simlpe_tencent_vocab = sorted(list(simlpe_tencent_vocab))

In [ ]:
# 储存相关数据
import numpy as np
simlpe_tencent_embedding = {emb[0]:np.array([float(e) for e in emb[1].split(" ")]) for emb in simlpe_tencent_embedding}
np.save("../user_data/data/dicts/simple_tencent_embedding.npy", simlpe_tencent_embedding)

with open("../user_data/data/dicts/simple_tencent_vocab.txt", "w", encoding="utf8") as w:
    w.write("\n".join(simlpe_tencent_vocab))


In [3]:
# 切词后，tencent_embedding中词的占比

import jieba
jieba.initialize("../user_data/data/dicts/simple_tencent_vocab.txt")

Building prefix dict from /home/heyilong/codes/chinese_medical_ner_submit/project/user_data/data/dicts/simple_tencent_vocab.txt ...


ValueError: invalid dictionary entry in /home/heyilong/codes/chinese_medical_ner_submit/project/user_data/data/dicts/simple_tencent_vocab.txt at Line 1: "

In [ ]:
all_words = set()
tencent_words = set()

for text in texts:
    for word in jieba.cut(text):
        all_words.add(word)
        if word in simlpe_tencent_vocab:
            tencent_words.add(word)

In [ ]:
len(all_words)

In [ ]:
len(tencent_words)

In [ ]:
"麻黄碱" in simlpe_tencent_vocab

In [ ]:
all_words - tencent_words

In [ ]:
"使用" in vocab